In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import shapefile as shp 

In [3]:
map_results_folder = '../map_results_onshore' 

In [4]:
def get_elcc_list(region,year,capacity,technology,add_on=None,scale_down=False):
    elcc_map_file = '%s/%s_%d_%d_MW_%s%s_results_map.csv' % (map_results_folder,region,year,capacity,technology,'' if add_on is None else '_'+add_on)
    elcc = pd.read_csv(elcc_map_file,index_col=0,header=0).values
    elcc = elcc[1::2,1::2] if scale_down else elcc
    elcc = elcc.flatten()
    elcc = elcc[np.logical_not(np.isnan(elcc))]
    return elcc

In [13]:
def plot_line(ax, elcc_1, elcc_2, elcc_3,label_xaxis=False,label_yaxis=False):

    sns.set_theme(style="whitegrid")

    for i in range(len(elcc_1)):
        ax.plot([1,2,3],[elcc_1[i],elcc_2[i],elcc_3[i]],c='r',lw=.5)

    ax.set_xticks([1,2,3])

    ax.grid(False)
    ax.set_ylim(bottom=0)

    #labels
    ax.set_xticklabels(['100 MW','500 MW','2000 MW'] if label_xaxis else [])
    ax.set_ylabel('ELCC (%)' if label_yaxis else '')

    return ax

In [14]:

fig, axs = plt.subplots(5,2,dpi=240, figsize=(7,10),constrained_layout=True)

year = 2019

i = 0

for region in ['basin','california','mountains','northwest','southwest']:
    for technology in ['solar','wind']:
        ax = axs[i//2,i%2]

        elcc_100 = get_elcc_list(region,year,100,technology)
        elcc_500 = get_elcc_list(region,year,500,technology,scale_down=True)
        elcc_2000 = get_elcc_list(region,year,2000,technology)
        
        ax = plot_line(ax,elcc_100,elcc_500,elcc_2000,
                        label_xaxis = True if i//2==4 else False,
                        label_yaxis = True if i%2==0 else False)

        title = '%s %s' % (region.capitalize(),technology.capitalize())
        ax.set_title(title)

        i +=1


img_name = '../gallery/figure_5_generator_size_%d' % year
plt.savefig(img_name)  
plt.close()           


In [5]:
def plot_box(ax, region, generator_size, elcc):

    sns.set_theme(style="whitegrid")

    sns.boxplot(x=region, y=elcc, hue=generator_size,ax=ax)
    ax.grid(False)
    
    return ax

In [6]:

fig, axs = plt.subplots(2,1,dpi=240, figsize=(7,8),constrained_layout=True)

year = 2019
for technology in ['solar','wind']:
    ax = axs[0 if technology=='solar' else 1]

    region_arr = np.array([])
    elcc = np.array([])
    generator_size = np.array([])

    for region in ['basin','california','mountains','northwest','southwest']:
        
            elcc_100 = get_elcc_list(region,year,100,technology)
            elcc_500 = get_elcc_list(region,year,500,technology,scale_down=True)
            elcc_2000 = get_elcc_list(region,year,2000,technology)
            
            generator_size = np.append(generator_size,['100 MW' for i in range(len(elcc_100))]).flatten()
            generator_size = np.append(generator_size,['500 MW' for i in range(len(elcc_100))]).flatten()
            generator_size = np.append(generator_size,['2000 MW' for i in range(len(elcc_100))]).flatten()
            elcc = np.append(elcc, np.array([elcc_100,elcc_500,elcc_2000]).flatten())
            region_arr = np.append(region_arr, np.array([region.capitalize() for i in range(len(elcc_100)*3)]))

    ax = plot_box(ax,region_arr, generator_size, elcc)

    title = '%s Generators' % (technology.capitalize())
    ax.set_title(title)
    ax.set_ylabel('ELCC (%)')

img_name = '../gallery/figure_5_generator_size_box_%d' % year
plt.savefig(img_name)  
plt.close()